In [2]:
import os
import scipy.io
import numpy as np

import xgboost as xgb
import lightgbm as lgb
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import train_test_split #to split the dataset for training and testing
from sklearn.svm import SVC   #for Support Vector Machine (SVM) Algorithm

from sklearn import preprocessing
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

from sklearn.neighbors import KNeighborsClassifier  # for K nearest neighbours
from sklearn.tree import DecisionTreeClassifier #for using Decision Tree Algoithm


import tensorflow as tf
from keras.utils.vis_utils import plot_model
from keras.preprocessing.sequence import TimeseriesGenerator

from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv1D, Conv2D, MaxPooling1D

from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


In [3]:

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [35]:


def load_data(folder_path):
    all_epochs = []
    all_triggers = []
    all_labels = []
    all_stimuli = []
    all_phase = []
    all_subjects = []

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".mat"):
                filepath = os.path.join(root, file)
                #print(filepath)
                
                mat = scipy.io.loadmat(filepath)
                # Replace 'data' with the actual key in your .mat files
                data = mat['epoch']  # Shape: (n_channels, n_times, n_epochs)
                
                if data is None:
                    print(f'File {file_path} is empty')
                    continue
                # Get the trigger type from the filename
                trigger_type = int(file.split('_type')[-1].split('.')[0])
                
                # Create a list of triggers based on the number of epochs
                #print(trigger_type)
                triggers = [trigger_type] * data.shape[2]
                
                # Get the label from the folder name
                label = 0 if 'Not_Blinded' in root else 1
                
                # Append data to the lists
                all_epochs.append(data)
                all_triggers.extend(triggers)
                all_labels.extend([label] * data.shape[2])
                stimulus, phase = get_stimulus_and_phase(filepath)
                all_stimuli.extend([stimulus]* data.shape[2])
                all_phase.extend([phase] * data.shape[2])
                #print(f"Stimulus: {stimulus}, Phase: {phase}")
                all_subjects.extend([get_subject_number(filepath)]* data.shape[2])
                

    # Concatenate lists into numpy arrays
    all_epochs = np.concatenate(all_epochs, axis=2)  # Shape: (n_channels, n_times, total_epochs)
    all_triggers = np.array(all_triggers)  # Shape: (total_epochs,)
    all_labels = np.array(all_labels)  # Shape: (total_epochs,)

    return all_epochs, all_triggers, all_labels, all_phase, all_stimuli, all_subjects



In [30]:
import re

def get_subject_number(file_name):
    match = re.search(r'IB_Sub_(\d+)_', file_name)
    if match:
        #print(int(match.group(1)))
        return int(match.group(1))
        
    else:
        #print("Found None Subj Num")
        return None  # or handle the case where no subject number is found
        
        #print(int(match.group(1)))


In [31]:
def get_stimulus_and_phase(file_name):
    stimulus = ""
    phase = ""

    if 'type210' in file_name:
        stimulus = 'face'
        phase = 'invisible'
    elif 'type220' in file_name:
        stimulus = 'face'
        phase = 'visible'
    elif 'type230' in file_name:
        stimulus = 'face'
        phase = 'reported'
    elif 'type211' in file_name:
        stimulus = 'house'
        phase = 'invisible'
    elif 'type221' in file_name:
        stimulus = 'house'
        phase = 'visible'
    elif 'type231' in file_name:
        stimulus = 'house'
        phase = 'reported'
    elif 'type212' in file_name:
        stimulus = 'noise'
        phase = 'invisible'
    elif 'type222' in file_name:
        stimulus = 'noise'
        phase = 'visible'
    elif 'type232' in file_name:
        stimulus = 'noise'
        phase = 'reported'
        
    #print(f"Stimulus: {stimulus}, Phase: {phase}")

    return stimulus, phase

# Example usage
#stimulus, phase = get_stimulus_and_phase(file_name)
#


In [32]:

# Load data from both folders
blinded_folder = "F:\\Triangulation\\Reed\IB\\DONE\\DONE\\Blinded\\Epochs_full"
not_blinded_folder = "F:\\Triangulation\\Reed\IB\\DONE\\DONE\\Not_Blinded\\Epochs_full"


In [36]:

epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b = load_data(blinded_folder)


In [37]:

epochs_not_blinded, triggers_not_blinded, labels_not_blinded, phase_n, stimuli_n, subjects_n  = load_data(not_blinded_folder)

# Combine data from both folders
epochs = np.concatenate([epochs_blinded, epochs_not_blinded], axis=2)
triggers = np.concatenate([triggers_blinded, triggers_not_blinded])
labels = np.concatenate([labels_blinded, labels_not_blinded])
phase = np.concatenate([phase_b, phase_n])
stim = np.concatenate([stimuli_b, stimuli_n])
subj = np.concatenate([subjects_b, subjects_n])

print("Epochs shape:", epochs.shape)
print("Triggers shape:", triggers.shape)
print("Labels shape:", labels.shape)

print("Epochs shape:", stim.shape)
print("Triggers shape:", phase.shape)
print("Labels shape:", subj.shape)

import pickle

# Save the data
with open('IB_eeg_data.pkl', 'wb') as f:
    pickle.dump((epochs, triggers, labels, phase, stim, subj), f)


Epochs shape: (63, 1250, 88707)
Triggers shape: (88707,)
Labels shape: (88707,)
Epochs shape: (88707,)
Triggers shape: (88707,)
Labels shape: (88707,)
